# Making A Neural Network From Scratch:

**Goal**: To implement a neural network using only numPy that has a one hidden layer and one output layer. We can then train our neural network on the MNIST data-set, and test it to see our accuracy. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chinese-mnist-digit-recognizer/chineseMNIST.csv
/kaggle/input/mnist-digit-recognizer/train.csv


I'm going to first import the mnist digits in csv form, every row represents an image and every column, a pixel, with the first column being the label of the number.

In [2]:
df = pd.read_csv("/kaggle/input/mnist-digit-recognizer/train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape # rows show how many pictures there are in the csv 

(42000, 785)

We are going to convert this DataFrame into a np.array, and use test_train_split() by sklearn. Then we want to take the **tranpose** of each array so that each array is actually an individual picture (which will be one of the 784 nodes).

In [4]:
x_train, x_test, y_train, y_test = train_test_split(np.array(df.iloc[:, 1:]), np.array(df[['label']]), test_size = .4)
x_train, x_test, y_train, y_test = x_train.T, x_test.T, y_train.T, y_test.T 
display(x_train[:, 0].shape, len(x_test[0]), y_train, y_test) # check configs
# n_xtrain (rows of pixels) = 784, m_xtrain (columns of items) = train/split

(784,)

16800

array([[5, 6, 9, ..., 5, 9, 8]])

array([[6, 2, 3, ..., 3, 0, 5]])

**Note**: Basically we want to make the train set an nxm matrix so we can left multiply it by a weight matrix to move from Rm -> R10 (10 is an arbitrary number for the first layer but makes computation easier). This will help to compute a linear combination with the weights and nodes 

# Initialization and Forward Propagation: 
1. weights
2. biases 
3. arrays for each layer:
    * input layer
    * first reLU layer (a function applied to each node)
    * second output layer (which will also have a softmax to find a probability distribution of numbers)
4. Make forward propagation 

We are also going to start with random weights and biases. Below is a diagram in LaTeX explaining the matrix multiplcation, where matrix $B$ is our input layer tranposed multiplied by our $A$ matrix, and the biases being added as matrix $C$. The result will be a $10 * m$ matrix, where $m$ is the number of training pictures.





$$\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1,784} \\
a_{21} & a_{22} & \cdots & a_{2,784} \\
\vdots & \vdots & \ddots & \vdots \\
a_{10,1} & a_{10,2} & \cdots & a_{10,784}
\end{bmatrix}
\times
\begin{bmatrix}
b_{11} & b_{12} & \cdots & b_{1m} \\
b_{21} & b_{22} & \cdots & b_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
b_{784,1} & b_{784,2} & \cdots & b_{784,m}
\end{bmatrix}
+
\begin{bmatrix}
c_{11} & c_{12} & \cdots & c_{1m} \\
c_{21} & c_{22} & \cdots & c_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
c_{10,1} & c_{10,2} & \cdots & c_{10,m}
\end{bmatrix}
$$


Here are the softmax function and ReLU function definitions:

$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^n e^{z_j}}
$

where $z_i$ is the i-th element of the input vector z. The softmax function will apply this to each $z_i$.

The softmax function is applied independently to each column of Z2. For a single column (logits for one image), softmax computes that equation.

$
\text{ReLU}(x) = \max(0, x)
$


In [5]:
def init_params(): 
    # populate a 10 x 784 matrix with random numbers, W1 is weight for first layer
    W1 = np.random.rand(10, 784) 
    # continue with biases, which should be a 10x1 vector, which we will broadcast
    # 10x1 -> 10xm so we can add the dot product + b
    b1 = np.random.rand(10, 1) 
    # second layer weights and biases
    W2 = np.random.rand(10, 10) 
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2
def reLU(Z): # return the same sized matrix, applied reLU
    for row in Z: 
        for i in range(row): 
            if row[i] < 0: 
                row[i] = 0;
    return Z
    # better solution use np.maximum(0, Z)
def softMax(Z): # input: 10 x m matrix
    return np.exp(Z) / np.sum(np.exp(Z)) 
    # collapses all rows to one and preserves columns
    
def forward_propagation(W1, b1, W2, b2, X): # X is the initial input matrix
    # Z1 will be the resulting matrix from matrix multiplication
    Z1 = W1.dot(X) + b1 
    # apply reLU to change from linear combination -> non-linear function
    A1 = ReLu(Z1)
    Z2 = W2.dot(A1) + b2 # 10 x m
    A2 = SoftMax(Z2)
